# Hypothetical Error-adjusted Measurement Outcome Demo

Note: the demo will not appear if you are looking at this on github because the widgets are dynamically created.
Note: this repo gives resampling counts of 100 for time constraints, but typically the sample count of more than 10,000 is more reliable.

## Imports and setup

In [1]:
# qiskit circuit & machine
from qiskit import QuantumCircuit
import qiskit_aer
from qiskit_ibm_runtime.fake_provider import FakeVigo
from qiskit.circuit.library import QFT
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# widget-related
import anywidget
import traitlets

# widget function
from state_uncertainty_vis import drawStateUncertainty, uncertainty_design
import numpy as np
import pandas as pd

# host setting for dev mode
_SS_ = "http://localhost:5175/"

class UncertaintyVis(anywidget.AnyWidget):
    _esm = _SS_ + "index.js?anywidget"
    vis_data = traitlets.Unicode().tag(sync=True)
    design = traitlets.Unicode().tag(sync=True)
    mode = traitlets.Unicode("uncertainty").tag(sync=True)

def getUncertaintyVis(job, transpiled, design={}, bootstrap=False, sampling_counts=10000, sample_size=None):
    prep = drawStateUncertainty(job, transpiled, bootstrap=bootstrap, sampling_counts=sampling_counts, sample_size=sample_size)
    vis_data = prep.toJSON()
    _design = uncertainty_design(design)
    __design = _design.toJSON()
    vis = UncertaintyVis(vis_data=vis_data, design=__design)
    return (vis, prep, _design)

## Write and run a circuit

In [2]:
qft = QFT(3).to_gate()

qcb = QuantumCircuit(3, 3)
qcb.h([0, 1, 2])
qcb.append(qft, [0, 1, 2])
qcb.measure([0,1,2], [0,1,2])

optimization_level = 1
backend = qiskit_aer.AerSimulator.from_backend(FakeVigo())
pass_manager = generate_preset_pass_manager(backend=backend, optimization_level=optimization_level, seed_transpiler=1)

qcb_trans = pass_manager.run(qcb)
job = backend.run(qcb_trans, shots=1000)

## Load a visualization

In [3]:
(unc, dt, design) = getUncertaintyVis(job, qcb_trans, design={"mark": "bar", "direction": "vertical"}, sampling_counts=100)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [4]:
unc

UncertaintyVis(design='{"mark": "bar", "direction": "vertical"}', vis_data='{"data": {"111": {"measure": 3, "p…

## Write and run a circuit (Case 2)

In [5]:
qft = QFT(3).to_gate()

qcx = QuantumCircuit(3, 3)
qcx.append(qft, [0, 1, 2])
qcx.measure([0,1,2], [0,1,2])

qcx_trans = pass_manager.run(qcx)
job = backend.run(qcx_trans, shots=1000)

## Load a visualization

In [6]:
(unc2, dt2, design2) = getUncertaintyVis(job, qcx_trans, design={"mark": "bar", "direction": "vertical"}, sampling_counts=100)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [7]:
unc2

UncertaintyVis(design='{"mark": "bar", "direction": "vertical"}', vis_data='{"data": {"110": {"measure": 125, …